In [3]:
!pip install opencv-python
!pip install pillow
!pip install ezdxf

In [4]:
#connect to google drive
import cv2
import numpy as np
from PIL import Image
import ezdxf
import os, sys
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
# Load the image and mask
id = "01"
inpath = '/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/'
inpath_ma = '/content/drive/Shareddrives/SmartCLT_DF2023/Scan2/GOOD_SCAN/'
outpath = '/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_0628/'

input_img_name = inpath + 'Copy of'+ id +'_image.jpg'
input_mask_name = inpath_ma + id +'_mask.jpg'
input_solu_name = inpath + 'Copy of'+ id +'_solution.png'


img = cv2.imread(input_img_name)
mask = cv2.imread(input_mask_name,0)
solution = cv2.imread(input_solu_name, cv2.IMREAD_GRAYSCALE)

In [6]:
# Increase brightness
bright_img = img + 50

# Ensure pixel values are within valid range
bright_img = np.clip(bright_img, 0, 255)


# Increase blue and green channels to make the image yellower
yellow_img = bright_img.copy()
'''
yellow_img[:, :, 0] = np.clip(yellow_img[:, :, 0] + 30, 0, 255)  # Blue channel
yellow_img[:, :, 1] = np.clip(yellow_img[:, :, 1] + 30, 0, 255)  # Green channel
'''
# Optionally, decrease red channel if the wood appears too red
yellow_img[:, :, 2] = np.clip(yellow_img[:, :, 2] - 20, 0, 255)  # Red channel

# Save the result
cv2.imwrite('bright_yellow_wood.jpg', yellow_img)


True

In [7]:
# Convert the image from RGB to BGR format (this is what OpenCV uses)
img_bgr = cv2.cvtColor(yellow_img, cv2.COLOR_RGB2BGR)

# Flip the solution vertically and horizontally
flipped_solu = cv2.flip(solution, -1)

# Blur solution to enhance edge detection
flipped_solu = cv2.GaussianBlur(flipped_solu, (5, 5), 0)

# Apply Canny edge detection
edges = cv2.Canny(flipped_solu,50,100)

# Find contours in the binary image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate over the contours and draw each one on the original image
buffer = 0.025
dy = -int(-3500/(1-2*buffer)*buffer)
dx = 0

for contour in contours:
  if contours.index(contour) == 1:
    contour += np.array([dx, dy])
    contour = contour.reshape((-1, 1, 2))
    cv2.polylines(img_bgr, [contour], isClosed=True, color=(40, 160, 40), thickness=2)

cv2.imwrite('output.png', img_bgr)

<ipython-input-7-2e7563816091>:22: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if contours.index(contour) == 1:


True

In [8]:
# Make sure your mask is a binary image (white and black only)
ret, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
mask = cv2.resize(mask, (img_bgr.shape[1], img_bgr.shape[0]))

# Erode the mask
kernel_size = 5  # You can adjust this value for more or less erosion
kernel = np.ones((kernel_size, kernel_size), np.uint8)
mask = cv2.erode(mask, kernel, iterations=1)

# Blur the mask
mask = cv2.GaussianBlur(mask,(5,5),0)

# Apply the mask to the image using bitwise 'and'
img_masked = cv2.bitwise_and(img_bgr, img_bgr, mask=mask)

# Convert the image to RGB for PIL
img_masked = cv2.cvtColor(img_masked, cv2.COLOR_BGR2RGB)

# Convert to a PIL Image
pil_img = Image.fromarray(img_bgr)

# Create an alpha mask
alpha_mask = Image.fromarray(mask)

# Convert black areas to transparent in the PIL image
pil_img.putalpha(alpha_mask)

# Save the image
pil_img.save('trans.png')

01 <class 'str'>
02 <class 'str'>
03 <class 'str'>
04 <class 'str'>
05 <class 'str'>
06 <class 'str'>
07 <class 'str'>
08 <class 'str'>
09 <class 'str'>
10 <class 'str'>
11 <class 'str'>
12 <class 'str'>
13 <class 'str'>
14 <class 'str'>
15 <class 'str'>
16 <class 'str'>
17 <class 'str'>
18 <class 'str'>
19 <class 'str'>
20 <class 'str'>
21 <class 'str'>
22 <class 'str'>
23 <class 'str'>
24 <class 'str'>
25 <class 'str'>
26 <class 'str'>
27 <class 'str'>
28 <class 'str'>
29 <class 'str'>
30 <class 'str'>
31 <class 'str'>
32 <class 'str'>
33 <class 'str'>
34 <class 'str'>


In [40]:
def crop_and_overlay(id):

  # 01_Load the image and mask
  inpath = '/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/'
  inpath_ma = '/content/drive/Shareddrives/SmartCLT_DF2023/Scan2/GOOD_SCAN/'
  outpath = '/content/drive/Shareddrives/SmartCLT_DF2023/Exhibition & Presentation/Imaging Process/image to dxf/'

  input_img_name = inpath + 'Copy of '+ id +'_image.jpg'
  input_mask_name = inpath_ma + id +'_mask.jpg'
  input_solu_name = inpath + 'Copy of '+ id +'_solution.png'

  out_name = outpath + id + '_overlay.png'

  print(input_img_name)

  img = cv2.imread(input_img_name)
  mask = cv2.imread(input_mask_name,0)
  solution = cv2.imread(input_solu_name, cv2.IMREAD_GRAYSCALE)


  #02_process image color



  # Increase brightness
  bright_img = img + 50

  # Ensure pixel values are within valid range
  bright_img = np.clip(bright_img, 0, 255)

  # Increase blue and green channels to make the image yellower
  yellow_img = bright_img.copy()

  # Optionally, decrease red channel if the wood appears too red
  yellow_img[:, :, 2] = np.clip(yellow_img[:, :, 2] - 20, 0, 255)  # Red channel



  # 03_overlay cutting curve
  # Convert the image from RGB to BGR format (this is what OpenCV uses)
  img_bgr = cv2.cvtColor(yellow_img, cv2.COLOR_RGB2BGR)

  # Flip the solution vertically and horizontally
  flipped_solu = cv2.flip(solution, 0)

  # Blur solution to enhance edge detection
  flipped_solu = cv2.GaussianBlur(flipped_solu, (5, 5), 0)

  # Apply Canny edge detection
  edges = cv2.Canny(flipped_solu,50,100)

  # Find contours in the binary image
  contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Iterate over the contours and draw each one on the original image
  buffer = 0.025
  dy = -int(-3500/(1-2*buffer)*buffer)
  dx = 0

  for contour in contours:
    if contours.index(contour) == 1:
      contour += np.array([dx, dy])
      contour = contour.reshape((-1, 1, 2))
      cv2.polylines(img_bgr, [contour], isClosed=True, color=(40, 160, 40), thickness=2)




  #04_crop with mask

  # Make sure your mask is a binary image (white and black only)
  img_bgr = img_bgr[int(3500/(1-2*buffer)*buffer):int(1-2*buffer*buffer+3500),0:606]
  ret, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
  mask = cv2.resize(mask, (img_bgr.shape[1], img_bgr.shape[0]))

  # Erode the mask
  kernel_size = 1  # You can adjust this value for more or less erosion
  kernel = np.ones((kernel_size, kernel_size), np.uint8)
  mask = cv2.erode(mask, kernel, iterations=1)

  # Blur the mask
  mask = cv2.GaussianBlur(mask,(1,1),0)

  # Apply the mask to the image using bitwise 'and'
  img_masked = cv2.bitwise_and(img_bgr, img_bgr, mask=mask)

  # Convert the image to RGB for PIL
  img_masked = cv2.cvtColor(img_masked, cv2.COLOR_BGR2RGB)

  # Convert to a PIL Image
  pil_img = Image.fromarray(img_bgr)

  # Create an alpha mask
  alpha_mask = Image.fromarray(mask)

  # Convert black areas to transparent in the PIL image
  pil_img.putalpha(alpha_mask)

  # Save the image
  # pil_img.save(out_name)
  pil_img.save('30_overlay.png')




In [28]:
for i in range(34):
    id=i+1
    if id<10:
      id='0'+str(id)
    else:
      id = str(id)
    crop_and_overlay(id)



/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 01_image.jpg


<ipython-input-26-cf2c4e48dc98>:59: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if contours.index(contour) == 1:


/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 02_image.jpg
/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 03_image.jpg
/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 04_image.jpg
/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 05_image.jpg
/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 06_image.jpg
/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 07_image.jpg
/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 08_image.jpg
/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 09_image.jpg
/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 10_image.jpg
/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 11_image.jpg
/content/drive/Share

ValueError: ignored

In [41]:
crop_and_overlay('30')

/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_FINAL/Copy of 30_image.jpg


<ipython-input-40-405e9823c042>:61: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if contours.index(contour) == 1:
